## Problems to solve: 
## 1. angle more than 360
## 2. remove indexing column
## 3. make node name only 1 node in the 1 intermediate and fly when it is flying
## 4. make new column to tell whether it is start or intermediate or end
## 5. fix global and relative wind direction so that it says none in wind speed 0

In [61]:
def iterate_through_dataset(**kwargs):
    for root in ["new_1Intermediate","new_2Intermediate"]:
        for path_no in listdir_nohidden(root):
            for wind_speed in listdir_nohidden(os.path.join(root,path_no)):
                for wind_angle in listdir_nohidden(os.path.join(root,path_no,wind_speed)):
                    for drone in listdir_nohidden(os.path.join(root,path_no,wind_speed,wind_angle)):
                        csv_path = os.path.join(root,path_no,wind_speed,wind_angle,drone)
                        kwargs["todo"](csv_path=csv_path,**kwargs)

In [32]:
def print_first_dataset(data):
    first_key = list(data.keys())[0]
    print(data[first_key])

In [23]:
import os
import pandas as pd

# loading all the dataset
data = {}

def listdir_nohidden(path):
    return [name for name in os.listdir(path) if name[0]!='.']

def store_dataset(csv_path,**kwargs):
    kwargs["data"][csv_path] = pd.read_csv(csv_path)

iterate_through_dataset(todo=store_dataset, data=data)
                    


In [34]:
print_first_dataset(data)

      Unnamed: 0  time.ms  stateEstimate.x  stateEstimate.y  stateEstimate.z  \
0              0        0         1.838746        -0.130389              0.0   
1              1      100         1.838710        -0.129682              0.0   
2              2      200         1.838939        -0.129794              0.0   
3              3      300         1.839083        -0.129884              0.0   
4              4      400         1.838939        -0.129908              0.0   
...          ...      ...              ...              ...              ...   
1068        1068   106800         0.728943         0.817873              0.0   
1069        1069   106900         0.728677         0.817726              0.0   
1070        1070   107000         0.728976         0.817790              0.0   
1071        1071   107100         0.728800         0.817754              0.0   
1072        1072   107200         0.728977         0.817772              0.0   

      stabilizer.roll  stabilizer.pitch

In [83]:
def iterate_over_rows(**kwargs):
    # create deep copy for the new cleaned data
    kwargs["new_data"][kwargs["csv_path"]] = kwargs["data"][kwargs["csv_path"]].copy()
    table = kwargs["new_data"][kwargs["csv_path"]]
    
    if "prepare_table_func" in kwargs:
        kwargs["prepare_table_func"](table=table,**kwargs) 

    # iterate over the row 
    for row_i in range(len(table)):
        kwargs["todo_row"](table, row_i, **kwargs)

def change_angle(table, row_i, **kwargs):
    for at in ["relative","global"]:
        if table[f"{at}_wind_direction"][row_i] != "None":
            angle = int(table[f"{at}_wind_direction"][row_i])
            if angle > 360:
                table[f"{at}_wind_direction"][row_i] = 720 - angle

# fixing angle more than 360
def fixing_angle_more_than_360(csv_path, **kwargs):
    iterate_over_rows(csv_path=csv_path, todo_row=change_angle, **kwargs)
        
# def fixing_angle_more_than_360(csv_path, **kwargs):

#     # create deep copy for the new cleaned data
#     kwargs["new_data"][csv_path] = kwargs["data"][csv_path].copy()
#     table = kwargs["new_data"][csv_path]
    
#     # iterate over the row 
#     for row_i in range(len(table)):
#         for at in ["relative","global"]:
#             if table[f"{at}_wind_direction"][row_i] != "None":
#                 angle = int(table[f"{at}_wind_direction"][row_i])
#                 if angle > 360:
#                     table[f"{at}_wind_direction"][row_i] = 720 - angle


 

In [ ]:
data_fixed_angle = {}
iterate_through_dataset(data=data, todo=fixing_angle_more_than_360, new_data = data_fixed_angle)

In [45]:
print_first_dataset(data_fixed_angle)

      Unnamed: 0  time.ms  stateEstimate.x  stateEstimate.y  stateEstimate.z  \
0              0        0         1.838746        -0.130389              0.0   
1              1      100         1.838710        -0.129682              0.0   
2              2      200         1.838939        -0.129794              0.0   
3              3      300         1.839083        -0.129884              0.0   
4              4      400         1.838939        -0.129908              0.0   
...          ...      ...              ...              ...              ...   
1068        1068   106800         0.728943         0.817873              0.0   
1069        1069   106900         0.728677         0.817726              0.0   
1070        1070   107000         0.728976         0.817790              0.0   
1071        1071   107100         0.728800         0.817754              0.0   
1072        1072   107200         0.728977         0.817772              0.0   

      stabilizer.roll  stabilizer.pitch

In [56]:
def removing_unnamed_column(csv_path, **kwargs):
    
    kwargs["new_data"][csv_path] = kwargs["data"][csv_path].copy()
    table = kwargs["new_data"][csv_path]
    unnamed_cols = [col for col in table.columns if "Unnamed" in col]

    for unnamed_col in unnamed_cols:
        table.drop(unnamed_col, axis=1, inplace=True)
    
removed_unnamed_column = {}
iterate_through_dataset(data=data_fixed_angle, todo=removing_unnamed_column, new_data=removed_unnamed_column)

In [57]:
print_first_dataset(removed_unnamed_column)

      time.ms  stateEstimate.x  stateEstimate.y  stateEstimate.z  \
0           0         1.838746        -0.130389              0.0   
1         100         1.838710        -0.129682              0.0   
2         200         1.838939        -0.129794              0.0   
3         300         1.839083        -0.129884              0.0   
4         400         1.838939        -0.129908              0.0   
...       ...              ...              ...              ...   
1068   106800         0.728943         0.817873              0.0   
1069   106900         0.728677         0.817726              0.0   
1070   107000         0.728976         0.817790              0.0   
1071   107100         0.728800         0.817754              0.0   
1072   107200         0.728977         0.817772              0.0   

      stabilizer.roll  stabilizer.pitch  stabilizer.yaw   pm.vbat  drone_id  \
0           -0.059909          0.522659        6.466911  4.070312         1   
1           -0.075598    

In [60]:
# path names
path_name = {}
path_name["1Intermediate"] = {}
path_name["2Intermediate"] = {}

# 1 intermediate
path_name["1Intermediate"]["path_1"] = {"drone_E3": ['A1','B2','F2'],
    "drone_E5": ['F1','B2','B1'],
    "drone_E7": ['A/E1','B2','C1']}
path_name["1Intermediate"]["path_2"] = {"drone_E3": ['F2','B2','A1'],
    "drone_E5": ['B1','B2','F1'],
    "drone_E7": ['C1','B2','A/E1']}
path_name["1Intermediate"]["path_3"] = {"drone_E3": ['C1','F1','J1'],
    "drone_E5": ['G1','F1','A/B1'],
    "drone_E7": ['A1','F1','E2']}
path_name["1Intermediate"]["path_4"] = {"drone_E3": ['C1','F1','J1'],
    "drone_E5": ['G1','F1','A/B1'],
    "drone_E7": ['A1','F1','E2']}
path_name["1Intermediate"]["path_5"] = {"drone_E3": ['C1','F2','F1'],
    "drone_E5": ['A1','F2','J1'],
    "drone_E7": ['E1B','F2','G1']}

# 2 intermediate
path_name["2Intermediate"]["path_1"] = {"drone_E3": ['A1','B2','F1','F2'],
    "drone_E5": ['J1','B2','F1','B1'],
    "drone_E7": ['A/E1','B2','F1','C1']}
path_name["2Intermediate"]["path_2"] = {"drone_E3": ['F2','B2','F1','A1'],
    "drone_E5": ['B1','B2','F1','J1'],
    "drone_E7": ['C1','B2','F1','A/E1']}
path_name["2Intermediate"]["path_3"] = {"drone_E3": ['F2','B2','F1','A/E1'],
    "drone_E5": ['B1','B2','F1','J1'],
    "drone_E7": ['G1','B2','F1','E2']}
path_name["2Intermediate"]["path_4"] = {"drone_E3": ['K1','B2','F1','A1'],
    "drone_E5": ['B1','B2','F1','E2'],
    "drone_E7": ['J1','B2','F1','A/E1']}
path_name["2Intermediate"]["path_5"] = {"drone_E3": ['B1','J1','F2','A1'],
    "drone_E5": ['G1','J1','F2','E2'],
    "drone_E7": ['K1','J1','F2','J2']}

In [88]:
# make node name only 1 node in the 1 intermediate and fly when it is flying

# return which node index is the drone currently in. Return "fly" if there is no difference in the waiting time
def determine_node_index(table, row_i):
    prev_waiting_time = table["waiting_time"][row_i-1][1:-1] # dont take the [] in the beginning and end
    curr_waiting_time = table["waiting_time"][row_i][1:-1]

    prev_waiting_time_list = [time for time in prev_waiting_time.split('|')]
    curr_waiting_time_list = [time for time in curr_waiting_time.split('|')]

    for i,prev,curr in zip([i for i in range(len(prev_waiting_time_list))],prev_waiting_time_list, curr_waiting_time_list):
        if prev != curr:
            return i

    # if exactly the same then it is flying
    return "fly"

# fix for only 1 row
def fix_node_name(table, row_i, **kwargs):
    
    # determine whether it is currently flying or in start or first_intermediate or second_intermediate or destination
    if row_i == 0: # assume that it will be in the starting node
        node_index = 0
    else:
        # look at the previous row and see if anything increase
        node_index = determine_node_index(table,row_i)
    
    if node_index == "fly":
        table["node_name"][row_i] = "fly"
    else:
        # get the node name
        path_info_list = kwargs["csv_path"].split('/')
        inter_name = path_info_list[0][4:]
        path_no = path_info_list[1]
        drone_name = path_info_list[4].split('.')[0]
        node_name = path_name[inter_name][path_no][drone_name][node_index]
        # update the table
        table["node_name"][row_i] = node_name
    

In [89]:
node_name_fixed = {}
iterate_through_dataset(data=removed_unnamed_column, todo=iterate_over_rows, todo_row=fix_node_name, new_data=node_name_fixed)

/tmp/ipykernel_120812/1554623607.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table["node_name"][row_i] = node_name
/tmp/ipykernel_120812/1554623607.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table["node_name"][row_i] = "fly"


In [74]:
print_first_dataset(node_name_fixed)

      time.ms  stateEstimate.x  stateEstimate.y  stateEstimate.z  \
0           0         1.838746        -0.130389              0.0   
1         100         1.838710        -0.129682              0.0   
2         200         1.838939        -0.129794              0.0   
3         300         1.839083        -0.129884              0.0   
4         400         1.838939        -0.129908              0.0   
...       ...              ...              ...              ...   
1068   106800         0.728943         0.817873              0.0   
1069   106900         0.728677         0.817726              0.0   
1070   107000         0.728976         0.817790              0.0   
1071   107100         0.728800         0.817754              0.0   
1072   107200         0.728977         0.817772              0.0   

      stabilizer.roll  stabilizer.pitch  stabilizer.yaw   pm.vbat  drone_id  \
0           -0.059909          0.522659        6.466911  4.070312         1   
1           -0.075598    

In [75]:
first_key = list(data.keys())[0]
print([a for a in data[first_key]["node_name"]  if a == "fly"])

[]


In [92]:
# make new column to tell whether it is start or intermediate or end

node_roles = {
    "1Intermediate":["start","intermediate","destination"],
    "2Intermediate":["start","first_intermediate","second_intermediate","destination"]
}
# fix for only 1 row
def fix_node_name(table, row_i, **kwargs):
    
    # determine whether it is currently flying or in start or first_intermediate or second_intermediate or destination
    if row_i == 0: # assume that it will be in the starting node
        node_index = 0
    else:
        # look at the previous row and see if anything increase
        node_index = determine_node_index(table,row_i)
    
    if node_index == "fly":
        table["curr_node_role"][row_i] = "fly"
    else:
        # get the inter_name
        path_info_list = kwargs["csv_path"].split('/')
        inter_name = path_info_list[0][4:]
        # update the curr_node_role column in the table based on node index and number of intermediate
        table["curr_node_role"][row_i] = node_roles[inter_name][node_index]

def add_curr_node_role_col(**kwargs):
    kwargs["table"]["curr_node_role"] = 0

curr_node_role_data = {}
iterate_through_dataset(
    data=node_name_fixed, 
    todo=iterate_over_rows, 
    todo_row=fix_node_name, 
    new_data=curr_node_role_data, 
    prepare_table_func=add_curr_node_role_col
    )

/tmp/ipykernel_120812/2060393170.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table["curr_node_role"][row_i] = node_roles[inter_name][node_index]
/home/alan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [85]:
print_first_dataset(curr_node_role_data)

      time.ms  stateEstimate.x  stateEstimate.y  stateEstimate.z  \
0           0         1.838746        -0.130389              0.0   
1         100         1.838710        -0.129682              0.0   
2         200         1.838939        -0.129794              0.0   
3         300         1.839083        -0.129884              0.0   
4         400         1.838939        -0.129908              0.0   
...       ...              ...              ...              ...   
1068   106800         0.728943         0.817873              0.0   
1069   106900         0.728677         0.817726              0.0   
1070   107000         0.728976         0.817790              0.0   
1071   107100         0.728800         0.817754              0.0   
1072   107200         0.728977         0.817772              0.0   

      stabilizer.roll  stabilizer.pitch  stabilizer.yaw   pm.vbat  drone_id  \
0           -0.059909          0.522659        6.466911  4.070312         1   
1           -0.075598    

In [137]:
# fix global and relative wind direction so that it says none in wind speed 0

# fix for 1 row
def fix_direction_speed_0_row(table, row_i, **kwargs):
    for wind_dir in ["global_wind_direction","relative_wind_direction"]:
        table[wind_dir][row_i] = "None"

def do_nothing(*args, **kwargs):
    pass

def fix_direction_speed_0(**kwargs):
    # check the wind speed
    wind_speed = kwargs["csv_path"].split('/')[2][-1]
    if wind_speed == '0':
        iterate_over_rows(todo_row=kwargs["todo_row_0"], **kwargs)
    else:
        iterate_over_rows(todo_row=do_nothing, **kwargs)



In [138]:
fixed_direction_speed_0 = {}
iterate_through_dataset(
    data=curr_node_role_data, 
    todo=fix_direction_speed_0, 
    todo_row_0=fix_direction_speed_0_row,
    new_data=fixed_direction_speed_0, 
    )

/tmp/ipykernel_120812/2485614251.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[wind_dir][row_i] = "None"


In [139]:
fixed_direction_speed_0["new_1Intermediate/path_1/wind_speed_0/wind_direction_None/drone_E3.csv"]

,time.ms,stateEstimate.x,stateEstimate.y,stateEstimate.z,stabilizer.roll,stabilizer.pitch,stabilizer.yaw,pm.vbat,drone_id,node_name,wind_speed,global_wind_direction,relative_wind_direction,travel_dist,waiting_time,curr_node_role
0,0,1.679049,1.154905,0.000000,-0.378442,1.000206,3.393783,4.054688,0,A1,0,None,None,0.000000,[6200|0|0],start
1,101,1.678216,1.154050,0.000000,-0.371548,1.027688,3.350090,4.058594,0,A1,0,None,None,0.001193,[6300|0|0],start
2,200,1.679045,1.154720,0.000000,-0.376665,1.002078,3.349531,4.054688,0,A1,0,None,None,0.002259,[6400|0|0],start
3,300,1.678883,1.154821,0.000000,-0.378358,1.002344,3.341466,4.054688,0,A1,0,None,None,0.002449,[6500|0|0],start
4,400,1.679315,1.154994,0.000000,-0.374177,0.983725,3.358741,4.054688,0,A1,0,None,None,0.002915,[6600|0|0],start
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,303500,0.664706,0.353158,0.012769,2.001527,0.439374,-2.988576,3.726562,0,fly,0,None,None,10.595045,[184000|108000|0],fly
3036,303600,0.660665,0.356399,0.003463,-0.934417,-1.918717,-4.148411,3.679688,0,fly,0,None,None,10.605696,[184000|108000|0],fly
3037,303700,0.656779,0.358986,0.000000,-2.497902,2.207164,-4.761312,3.605469,0,fly,0,None,None,10.611508,[184000|108000|0],fly
3038,303800,0.652817,0.365781,0.000000,1.837420,-2.413095,-5.601729,3.705078,0,fly,0,None,None,10.619375,[184000|108000|0],fly


In [117]:
# write into csv file
def write_csv(**kwargs):
    new_path = kwargs["csv_path"].replace("new","cleaned")
    kwargs["data"][kwargs["csv_path"]].to_csv(new_path, index=False)

iterate_through_dataset(data=fixed_direction_speed_0, todo=write_csv)